In [ ]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timezone, timedelta

In [ ]:
captures = pd.read_csv('bolaks_pen_id_88_2020-02-10_2020-03-10.csv')

In [ ]:
pairs = pd.read_csv('bolaks_data_pairs.csv')

In [ ]:
captures.head(20)

In [ ]:
captures.captured_at = pd.to_datetime(captures['captured_at'])
pairs.p1_captured_at = pd.to_datetime(pairs['p1_captured_at'])
pairs.p2_captured_at = pd.to_datetime(pairs['p2_captured_at'])




In [ ]:
gaps = captures['captured_at'].diff()

np.mean(gaps < datetime.timedelta(seconds=0.5)), np.mean(gaps < datetime.timedelta(seconds=1.5)), np.mean(gaps < datetime.timedelta(seconds=2.5))

In [ ]:
for i in np.arange(11, 20):
    daily_captures = (captures['captured_at'] > datetime(2020, 2, i, tzinfo=timezone.utc)) & (captures['captured_at'] < datetime(2020, 2, i + 1, tzinfo=timezone.utc))
    daily_pairs = (pairs['p1_captured_at'] > datetime(2020, 2, i, tzinfo=timezone.utc)) & (pairs['p1_captured_at'] < datetime(2020, 2, i + 1, tzinfo=timezone.utc))

    print(i, np.sum(daily_captures), np.sum(daily_pairs))

In [ ]:
found_pairs = []

for i in np.arange(11, 20):
    mask = (captures['captured_at'] > datetime(2020, 2, i, tzinfo=timezone.utc)) & (captures['captured_at'] < datetime(2020, 2, i + 1, tzinfo=timezone.utc))

    old_captured_ats = []
    old_ids = []
    old_tails = []
    old_eyes = []

    count = 0
    correct_count = 0

    for index, row in captures[mask].iterrows():
        current_captured_at = row.captured_at

        # Delete old captures
        old_captured_at_indices = [i for i, ca in enumerate(old_captured_ats) if current_captured_at - ca < timedelta(seconds=10)]
        old_captured_ats = [ca for ca in old_captured_ats if current_captured_at - ca < timedelta(seconds=10)]
        old_ids = [_id for j, _id in enumerate(old_ids) if j in old_captured_at_indices ]
        old_tails = [row for j, row in enumerate(old_tails) if j in old_captured_at_indices ]
        old_eyes = [row for j, row in enumerate(old_eyes) if j in old_captured_at_indices ]

        # Check captures
        left_crops = json.loads(row['annotation'].replace("\'", "\""))['leftCrop']
        left_tail = [crop for crop in left_crops if crop['keypointType'] == 'ANAL_FIN'][0]
        left_eye = [crop for crop in left_crops if crop['keypointType'] == 'EYE'][0]
    #     right_crops = json.loads(row['annotation'].replace("\'", "\""))['rightCrop']
    #     right_eye = [crop for crop in right_crops if crop['keypointType'] == 'EYE'][0]

        for i, old_eye in enumerate(old_eyes):
            ca = old_captured_ats[i]
            old_tail = old_tails[i]
            if left_tail['xFrame'] < left_eye['xFrame']:
                horizontal_condition = old_tail['xFrame'] < old_eye['xFrame'] and old_eye['xFrame'] < left_eye['xFrame'] and old_tail['xFrame'] < left_tail['xFrame']
                vertical_condition = np.abs(old_eye['yFrame'] - left_eye['yFrame']) < 100

                if horizontal_condition and vertical_condition:
                    count = count + 1

                    if np.sum((pairs.p2 == index) & (pairs.p1 == old_ids[i])) > 0:
                        correct_count = correct_count + 1
                    else:
                        pass
                    
                    found_pairs.append([old_ids[i], index, np.sum((pairs.p2 == index) & (pairs.p1 == old_ids[i])) > 0, (current_captured_at - ca).total_seconds(), np.abs(old_eye['xFrame'] - left_eye['xFrame']), np.abs(old_eye['yFrame'] - left_eye['yFrame']), np.abs(old_tail['xFrame'] - left_tail['xFrame']), np.abs(old_tail['yFrame'] - left_tail['yFrame']), old_eye['xFrame'], old_eye['yFrame'], old_tail['xFrame'], old_tail['yFrame'], left_eye['xFrame'], left_eye['yFrame'], left_tail['xFrame'], left_tail['yFrame']])
                        #print(index, old_ids[i])
                #print(index, old_ids[i])
                #print(left_eye['xFrame'])
            else:
                horizontal_condition = old_tail['xFrame'] > old_eye['xFrame'] and old_eye['xFrame'] > left_eye['xFrame'] and old_tail['xFrame'] > left_tail['xFrame']
                vertical_condition = np.abs(old_eye['yFrame'] - left_eye['yFrame']) < 100

                if horizontal_condition and vertical_condition:
                    count = count + 1

                    if np.sum((pairs.p2 == index) & (pairs.p1 == old_ids[i])) > 0:
                        correct_count = correct_count + 1
                    else:
                        pass
                    
                    found_pairs.append([old_ids[i], index, np.sum((pairs.p2 == index) & (pairs.p1 == old_ids[i])) > 0, (current_captured_at - ca).total_seconds(), np.abs(old_eye['xFrame'] - left_eye['xFrame']), np.abs(old_eye['yFrame'] - left_eye['yFrame']), np.abs(old_tail['xFrame'] - left_tail['xFrame']), np.abs(old_tail['yFrame'] - left_tail['yFrame']), old_eye['xFrame'], old_eye['yFrame'], old_tail['xFrame'], old_tail['yFrame'], left_eye['xFrame'], left_eye['yFrame'], left_tail['xFrame'], left_tail['yFrame']])

        old_captured_ats.append(current_captured_at)
        old_ids.append(index)
        old_tails.append(left_tail)
        old_eyes.append(left_eye)

    print(count, correct_count)


In [ ]:
import csv

with open('duplicate_detections.csv', 'w') as outcsv:   
    writer = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
    writer.writerow(['p1', 'p2', 'paired_before', 'timedelta', 'eye_to_eye_xFrame', 'eye_to_eye_yFrame', 'tail_to_tail_xFrame', 'tail_to_tail_yFrame', 'old_eye_xFrame', 'old_eye_yFrame', 'old_tail_xFrame', 'old_tail_yFrame', 'left_eye_xFrame', 'left_eye_yFrame', 'left_tail_xFrame', 'left_tail_yFrame'])
    for item in found_pairs:
        writer.writerow(item)

In [ ]:
duplicate_detections = pd.read_csv('duplicate_detections.csv')

In [ ]:
duplicate_detections.head()

In [ ]:
duplicate_detections.tail()

In [ ]:
np.mean(duplicate_detections['eye_to_eye_xFrame'] - duplicate_detections['tail_to_tail_xFrame'] < 100)

In [ ]:
#pairs.sort_values(by=['p1']).tail()

In [ ]:
np.mean(pairs.p1_captured_at == pairs.p2_captured_at)

In [ ]:
wrong_pair = (53, 52)

np.sum((pairs.p1 == wrong_pair[0]) & (pairs.p2 == wrong_pair[1]))

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url=captures.ix[wrong_pair[0], 'left_crop_url'])

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url=captures.ix[wrong_pair[1], 'left_crop_url'])

In [ ]:
#json.loads(captures.ix[13050, 'annotation'].replace("\'", "\""))['leftCrop']

In [ ]:
json.loads(captures.ix[13051, 'annotation'].replace("\'", "\""))['leftCrop']

In [ ]:
captures.ix[101]